# Final Project: Web Service APIs
## Token aquisition notebook
by Lin Ma and Khanh Tran

   We worked with **Spotify** as our API provider to explore data in this final project. 
**Spotify** is a music streaming platform developed by Swedish company Spotify Technology. It is a API provider that require Authentication and Delegated Authorization (OAuth 2) to enable access to data that is unavailiable to anonymous user.
Before we start coding, we went to Spotify's Developer homepage (https://developer.spotify.com/) and created a third party Application called **Happy Penguin**. For the redirect url  we constructed a basic website on github.
This notebook include every setps and code we have done to get the access token from our users, including get the redirect url, create the access url, get the user code, use the codes to aquire tokens. 

To be more specific, as the third party application, we requested authorization from Spotify Developer to access data, we stored our client id, cliend secret, response type, state and scope in a file named cred.json then map it with spotify account service. Then by having the users click in our access url and authorize us with different scopes by login their spotify account. 
By enter the user code mannualy, we mapped users' token information and stored it in a dictionary.


### Challenges & Success
One of the challenge we faced at the first was giving the insufficent scope and have the users authorized us with it.
Orignially, the scope we have is:
- user-top-read: Get a User's Top Artists and Tracks
However, we realized that it is not enough to find a few interesting question to answer. And we didn't have much scope at first place is also becasue we don't know the valid way to have motiple scope all at once. So we after we figued that out by doing a deep search on spotify developer webpage, we decided to added scopes as many as possible then we can decide which one to pay special attention with later when reaching the data aquicition step. We have to asked our users to authorize us again, now the scope we have include:

- user-follow-read: Get User's Followed Artists
- user-library-read:Get Current User's Saved Albums &Get a User's Saved Tracks
- user-top-read : Get a User's Top Artists and Tracks
- user-read-email:Get Current User's Profile (Because we didn't get through the IRB process, we decided not to use any data from this scope in the end)
- user-read-birthdate: Get Current User's Profile (Because we didn't get through the IRB process, we decided not to use any data from this scope in the end)
- user-read-recently-played: Get Current User's Recently Played Tracks
- user-read-private: Get a User's Available Devices & Get Information About The User's Current Playback & Get the User's Currently Playing Track(Because we didn't get through the IRB process, we decided not to use any data from this scope in the end)


Other then that, the user codes always expire in 10 mins. So when we first try to get all the users' code entered and exchange for token all at once, some of the code already expired. Therefore, we have to enter the code and exchaneg for token for a few user at a time to fit in the ten minues window. 


In [49]:
import json
import requests

In [50]:
def getURL(creds):
    """ This function takes in an url, parses the data into an element tree
    and returns the root element of that tree.
    Parameters:
        table_url: a given url
    Return: root element of etree
    """
    protocol = "https"
    location = "accounts.spotify.com"
    auth_resource = "/authorize"

    authurl_fmt = "{}://{}{}"
    authurl = authurl_fmt.format(protocol, location, auth_resource)
    urlquery = {}
    urlquery['client_id'] = creds['spotify']['appid']
    urlquery['redirect_uri'] = creds['spotify']['redirect_uri']
    urlquery['response_type'] = 'code'
    urlquery['state'] = '1234567890'
    urlquery['scope'] = ["user-follow-read user-library-read user-top-read user-read-email user-read-birthdate user-read-recently-played user-read-private"]
    session = requests.Session()
    p = requests.Request('GET', authurl, params=urlquery).prepare()
    return p.url

In [51]:
def getCodeMap():
    """ This function open a json file and lode it in this notebook as codemap
    Parameters:
        None
    Return: codemap: a json lode file for user maps with their code
    """
    with open("user_code.json", "r") as file1:
        codemap = json.load(file1)
    return codemap

In [52]:
def createAccessURL():
    """ This function consturct an access url for requesting authorization to access data
    Parameters:
        None
    Return: accessurl: A valid url that use for requesting authorization to access data
    """
    protocol = "https"
    location = "accounts.spotify.com"
    auth_resource = "/api/token"

    access_fmt = "{}://{}{}"
    accessurl = access_fmt.format(protocol, location, auth_resource)
    return accessurl

In [53]:
def getToken(accessurl, codemap, tokenmap, creds, user, code):
    """ This function takes in Accessurl, codemap, tokenmap, creds, user, code to exchange token from the api provider
    Parameters:
        accessurl: the valid accessurl that has been construct in the previous function
        codemap: the loaded json file that are used to mape user with it's code 
        tokenmap: the toke map json file which includes the mapping for all the users
        creds: the credentials json file for the third party application
        user: users form code map
        code: the code mapping with each user from the code map
    Return: tokenmap: the exchanged token for each of the user 
    """
    urlquery = {}
    urlquery['redirect_uri'] = creds['spotify']['redirect_uri']
    urlquery['grant_type'] = "authorization_code"
    urlquery['code'] = code
    resp = requests.post(accessurl, data=urlquery, auth = (creds['spotify']['appid'], creds['spotify']['appsecret'])) 
    retval = resp.json()
    if resp.status_code == 200:
        tokenmap[user] = {}
        if 'access_token' in retval:
            tokenmap[user]['access_token'] =retval['access_token']
        if 'token_type' in retval:
            tokenmap[user]['token_type'] =retval['token_type']
        if 'refresh_token' in retval:
            tokenmap[user]['refresh_token'] =retval['refresh_token']
        if 'expires_in' in retval:
            tokenmap[user]['expires_in'] =retval['expires_in']
        if 'scope' in retval:
            tokenmap[user]['scope'] =retval['scope']
    return tokenmap

In [54]:
with open("creds.json", "r") as file:
    creds = json.load(file)
url = getURL(creds)
print(url)
accessurl = createAccessURL()
tokenmap = {}

https://accounts.spotify.com/authorize?client_id=393ab8e9d5804dd3a3cbe6ed01efb54c&redirect_uri=https%3A%2F%2Flinma619.github.io%2FCS181Project%2F&response_type=code&state=1234567890&scope=user-follow-read+user-library-read+user-top-read+user-read-email+user-read-birthdate+user-read-recently-played+user-read-private


In [55]:
codemap = getCodeMap()
for user in codemap:
    if codemap[user] != {}: 
        getToken(accessurl, codemap, tokenmap, creds, user, codemap[user])

In [56]:
with open("user_token.json", "w") as filex:
    json.dump(tokenmap, filex)